In [11]:
# QUESTION
#
# For this task you will need to train a neural network
# to predict sunspot activity using the Sunspots.csv dataset.
# Your neural network must  have an MAE
# of 0.12 or less on the normalized dataset for top marks.
# Code for normalizing the data is provided and should not be changed.
# At the bottom of this file, we provide  some testing
# code should you want to check your model.

# Note: Do not use lambda layers in your model, they are not supported
# on the grading infrastructure.
# <번역> 
# 이 작업을 위해서는 신경망을 훈련시켜야합니다.
# Sunspots.csv 데이터 세트를 사용하여 흑점 활동을 예측합니다.
# 상위 마크에 대한 정규화 된 데이터 세트에서 0.12 이하의 MAE가 있어야합니다.
# 데이터 정규화를위한 코드가 제공되며 변경해서는 안됩니다.
# 당신의 모델을 체크하려면, 이 파일의 맨 아래에 몇 가지 테스팅 코드가 있습니다.
# 참고 : 모델에서 람다 레이어를 사용하지 마십시오. 채점 인프라에서 지원되지 않습니다.
 


import csv
import tensorflow as tf
import numpy as np
import urllib
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, LSTM 
from tensorflow.keras.optimizers import SGD, RMSprop
from tensorflow.keras.losses import Huber
from tensorflow.keras.callbacks import ModelCheckpoint


# DO NOT CHANGE THIS CODE
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)


def solution_model():
    url = 'https://storage.googleapis.com/download.tensorflow.org/data/Sunspots.csv'
    urllib.request.urlretrieve(url, 'sunspots.csv')

    time_step = []
    sunspots = []

    with open('sunspots.csv') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        next(reader)
        for row in reader:
            sunspots.append(float(row[2]))   # YOUR CODE HERE
            time_step.append(int(row[0]))  # YOUR CODE HERE

    series =  np.array(sunspots)# YOUR CODE HERE

    # DO NOT CHANGE THIS CODE
    # This is the normalization function
    min = np.min(series)
    max = np.max(series)
    series -= min
    series /= max
    time = np.array(time_step)

    # The data should be split into training and validation sets at time step 3000
    # DO NOT CHANGE THIS CODE
    split_time = 3000


    time_train = time[:split_time]# YOUR CODE HERE
    x_train = series[:split_time]# YOUR CODE HERE
    time_valid = time[split_time:]# YOUR CODE HERE
    x_valid = series[split_time:] # YOUR CODE HERE

    # DO NOT CHANGE THIS CODE
    window_size = 30
    batch_size = 32
    shuffle_buffer_size = 1000

    train_set = windowed_dataset(x_train, window_size=window_size, batch_size=batch_size, shuffle_buffer=shuffle_buffer_size)
    valid_set = windowed_dataset(x_valid, window_size=window_size, batch_size=batch_size, shuffle_buffer=shuffle_buffer_size)

    model = tf.keras.models.Sequential([
        Conv1D(60, kernel_size=5, padding='causal', activation='relu', input_shape=[None,1]),
        LSTM(60,return_sequences=True),
        LSTM(60, return_sequences=True),
        Dense(30, activation='relu'),
        Dense(10, activation='relu'),                                  
        # YOUR CODE HERE. Whatever your first layer is, the input shape will be [None,1] when using the Windowed_dataset above, depending on the layer type chosen
        tf.keras.layers.Dense(1)
    ])
    opt = SGD(lr=1e-5, momentum=0.9)
    model.compile(loss=Huber(), optimizer=opt, metrics=['mae'])
    
    checkpoint_path = 'sunset_checkpoint1.ckpt'
    checkpoint = ModelCheckpoint(filepath = checkpoint_path, save_weights_only=True, save_best_only=True, 
                                monitor='val_loss', verbose=1)
    model.fit(train_set, validation_data = (valid_set), epochs=100, callbacks=[checkpoint])
    model.load_weights(checkpoint_path)
    # YOUR CODE HERE TO COMPILE AND TRAIN THE MODEL
    return model


# Note that you'll need to save your model as a .h5 like this.
# When you press the Submit and Test button, this .h5 model will be
# sent to the testing infrastructure for scoring.

# You must use the Submit and Test button to submit your model
# at least once in each category before you finally submit your exam.

if __name__ == '__main__':
    model = solution_model()
    model.save("cat5_sunspots(typeB).h5")



# THIS CODE IS USED IN THE TESTER FOR FORECASTING. IF YOU WANT TO TEST YOUR MODEL
# BEFORE UPLOADING YOU CAN DO IT WITH THIS
#def model_forecast(model, series, window_size):
#    ds = tf.data.Dataset.from_tensor_slices(series)
#    ds = ds.window(window_size, shift=1, drop_remainder=True)
#    ds = ds.flat_map(lambda w: w.batch(window_size))
#    ds = ds.batch(32).prefetch(1)
#    forecast = model.predict(ds)
#    return forecast


#window_size = # YOUR CODE HERE
#rnn_forecast = model_forecast(model, series[..., np.newaxis], window_size)
#rnn_forecast = rnn_forecast[split_time - window_size:-1, -1, 0]

#result = tf.keras.metrics.mean_absolute_error(x_valid, rnn_forecast).numpy()

## To get the maximum score, your model must have an MAE OF .12 or less.
## When you Submit and Test your model, the grading infrastructure
## converts the MAE of your model to a score from 0 to 5 as follows:

#test_val = 100 * result
#score = math.ceil(17 - test_val)
#if score > 5:
#    score = 5

#print(score)

Epoch 1/100
     90/Unknown - 1s 11ms/step - loss: 0.0354 - mae: 0.2048
Epoch 00001: val_loss improved from inf to 0.02504, saving model to sunset_checkpoint1.ckpt
93/93 [==============================] - 2s 18ms/step - loss: 0.0356 - mae: 0.2052 - val_loss: 0.0250 - val_mae: 0.1748
Epoch 2/100
91/93 [============================>.] - ETA: 0s - loss: 0.0350 - mae: 0.2022
Epoch 00002: val_loss improved from 0.02504 to 0.02446, saving model to sunset_checkpoint1.ckpt
93/93 [==============================] - 1s 12ms/step - loss: 0.0349 - mae: 0.2020 - val_loss: 0.0245 - val_mae: 0.1718
Epoch 3/100
92/93 [============================>.] - ETA: 0s - loss: 0.0344 - mae: 0.1994
Epoch 00003: val_loss improved from 0.02446 to 0.02391, saving model to sunset_checkpoint1.ckpt
93/93 [==============================] - 1s 12ms/step - loss: 0.0343 - mae: 0.1990 - val_loss: 0.0239 - val_mae: 0.1690
Epoch 4/100
90/93 [============================>.] - ETA: 0s - loss: 0.0335 - mae: 0.1960
Epoch 00004: v

Epoch 28/100
91/93 [============================>.] - ETA: 0s - loss: 0.0235 - mae: 0.1563
Epoch 00028: val_loss improved from 0.01537 to 0.01515, saving model to sunset_checkpoint1.ckpt
93/93 [==============================] - 1s 12ms/step - loss: 0.0234 - mae: 0.1561 - val_loss: 0.0151 - val_mae: 0.1301
Epoch 29/100
91/93 [============================>.] - ETA: 0s - loss: 0.0231 - mae: 0.1549
Epoch 00029: val_loss improved from 0.01515 to 0.01494, saving model to sunset_checkpoint1.ckpt
93/93 [==============================] - 1s 13ms/step - loss: 0.0231 - mae: 0.1551 - val_loss: 0.0149 - val_mae: 0.1293
Epoch 30/100
91/93 [============================>.] - ETA: 0s - loss: 0.0228 - mae: 0.1540
Epoch 00030: val_loss improved from 0.01494 to 0.01473, saving model to sunset_checkpoint1.ckpt
93/93 [==============================] - 1s 12ms/step - loss: 0.0229 - mae: 0.1542 - val_loss: 0.0147 - val_mae: 0.1286
Epoch 31/100
91/93 [============================>.] - ETA: 0s - loss: 0.0226 - 

Epoch 55/100
90/93 [============================>.] - ETA: 0s - loss: 0.0182 - mae: 0.1398
Epoch 00055: val_loss improved from 0.01144 to 0.01136, saving model to sunset_checkpoint1.ckpt
93/93 [==============================] - 1s 12ms/step - loss: 0.0182 - mae: 0.1400 - val_loss: 0.0114 - val_mae: 0.1167
Epoch 56/100
91/93 [============================>.] - ETA: 0s - loss: 0.0179 - mae: 0.1388
Epoch 00056: val_loss improved from 0.01136 to 0.01128, saving model to sunset_checkpoint1.ckpt
93/93 [==============================] - 1s 12ms/step - loss: 0.0181 - mae: 0.1397 - val_loss: 0.0113 - val_mae: 0.1164
Epoch 57/100
91/93 [============================>.] - ETA: 0s - loss: 0.0180 - mae: 0.1395
Epoch 00057: val_loss improved from 0.01128 to 0.01120, saving model to sunset_checkpoint1.ckpt
93/93 [==============================] - 1s 12ms/step - loss: 0.0180 - mae: 0.1394 - val_loss: 0.0112 - val_mae: 0.1161
Epoch 58/100
91/93 [============================>.] - ETA: 0s - loss: 0.0178 - 

Epoch 82/100
91/93 [============================>.] - ETA: 0s - loss: 0.0159 - mae: 0.1347
Epoch 00082: val_loss improved from 0.01001 to 0.00998, saving model to sunset_checkpoint1.ckpt
93/93 [==============================] - 1s 12ms/step - loss: 0.0159 - mae: 0.1349 - val_loss: 0.0100 - val_mae: 0.1122
Epoch 83/100
91/93 [============================>.] - ETA: 0s - loss: 0.0158 - mae: 0.1347
Epoch 00083: val_loss improved from 0.00998 to 0.00995, saving model to sunset_checkpoint1.ckpt
93/93 [==============================] - 1s 12ms/step - loss: 0.0159 - mae: 0.1348 - val_loss: 0.0100 - val_mae: 0.1121
Epoch 84/100
91/93 [============================>.] - ETA: 0s - loss: 0.0158 - mae: 0.1347
Epoch 00084: val_loss improved from 0.00995 to 0.00993, saving model to sunset_checkpoint1.ckpt
93/93 [==============================] - 1s 12ms/step - loss: 0.0158 - mae: 0.1347 - val_loss: 0.0099 - val_mae: 0.1121
Epoch 85/100
91/93 [============================>.] - ETA: 0s - loss: 0.0157 - 

In [12]:
# <참고용코드-모델의 성능을 테스트하기 위한 코드 5등급 ok> #
# 이부분은 절대 답으로 포함시키지 마세요 #
# THIS CODE IS USED IN THE TESTER FOR FORECASTING. IF YOU WANT TO TEST YOUR MODEL
# BEFORE UPLOADING YOU CAN DO IT WITH THIS
import math
url = 'https://storage.googleapis.com/download.tensorflow.org/data/Sunspots.csv'
urllib.request.urlretrieve(url, 'sunspots.csv')

time_step = []
sunspots = []

with open('sunspots.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        sunspots.append(float(row[2]))   # YOUR CODE HERE
        time_step.append(int(row[0]))  # YOUR CODE HERE

series =  np.array(sunspots)# YOUR CODE HERE

# DO NOT CHANGE THIS CODE
# This is the normalization function
min = np.min(series)
max = np.max(series)
series -= min
series /= max
time = np.array(time_step)
split_time = 3000
time_train = time[:split_time]# YOUR CODE HERE
x_train = series[:split_time]# YOUR CODE HERE
time_valid = time[split_time:]# YOUR CODE HERE
x_valid = series[split_time:] # YOUR CODE HERE

# DO NOT CHANGE THIS CODE
window_size = 30
batch_size = 32
shuffle_buffer_size = 1000
    
def model_forecast(model, series, window_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(32).prefetch(1)
    forecast = model.predict(ds)
    return forecast

# window_size =  30# YOUR CODE HERE
rnn_forecast = model_forecast(model, series[..., np.newaxis], window_size)
rnn_forecast = rnn_forecast[split_time - window_size:-1, -1, 0]

result = tf.keras.metrics.mean_absolute_error(x_valid, rnn_forecast).numpy()

# To get the maximum score, your model must have an MAE OF .12 or less.
# When you Submit and Test your model, the grading infrastructure
# converts the MAE of your model to a score from 0 to 5 as follows:

test_val = 100 * result
score = math.ceil(17 - test_val)
if score > 5:
    score = 5

print(score)

5
